## Starbucks challenge

This project has been divided into three notebook arts:
- Data pre-processing and filtering
- Data analysis
- Model building, deploying and evaluating


#### (a) Data pre-processing 

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

print(portfolio.shape, profile.shape, transcript.shape)

(10, 6) (17000, 5) (306534, 4)


We are given the following three tables: **portfolio** with a shape of (10,6), **profile** with a shape of (17000,5) and **transcript** with a shape of (306534,4).

#### Peek into the three tables

#### (a) portfolio table

In [2]:
portfolio.head(5)

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [3]:
portfolio.shape

(10, 6)

Since there are only 10 records in the data, we are interested to know how many unique offers are present.

In [4]:
print('Count of unique offers: ',len(portfolio.id.unique()))

Count of unique offers:  10


#### (b) profile table

In [5]:
profile.head(5)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [6]:
profile.shape

(17000, 5)

In [7]:
print('Count of unique people: ', len(profile.id.unique()))

Count of unique people:  17000


Some Observations:
- Age has 118 as value which seems wierd.
- income column has Null values.

We want to check if the people who are supposedly 118 years also have null income. If this is the case, we will drop such rows.

In [8]:
profile.loc[(profile['age']==118) & (profile['income'].isnull()==True)].shape

(2175, 5)

There are 2175 such records. It is safe to drop them.

In [9]:
profile.dropna(inplace=True)

In [10]:
print('New shape of the profile data: ', profile.shape)

New shape of the profile data:  (14825, 5)


In [11]:
#changing id column in profile data to be more specif and meaningful.
profile.rename(columns={'id':'customerid'}, inplace=True)
print(profile.columns)

Index(['age', 'became_member_on', 'gender', 'customerid', 'income'], dtype='object')


#### (c) transcript table 
**Note:** Since this dataset is quite tricky to clean, we will do that in later steps. First we will only look at some structural aspects such as shape, contents etc.

In [12]:
transcript.shape

(306534, 4)

In [13]:
transcript.tail(5)

,event,person,time,value
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,714,{'amount': 1.5899999999999999}
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,714,{'amount': 9.53}
306531,transaction,a00058cf10334a308c68e7631c529907,714,{'amount': 3.61}
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,714,{'amount': 3.5300000000000002}
306533,transaction,c02b10e8752c4d8e9b73f918558531f7,714,{'amount': 4.05}


In [14]:
transcript.isna().any()

event     False
person    False
time      False
value     False
dtype: bool

Some Observations: <br>
- value column has dictionaries. In the data cleaning step, we will pre-process this column.
- time column has the values in number of hours. For simplicity, we will turn it into number of days in the later step.
- there are no null values.

### Cleaning the tables
#### (a) portfolio
- one-hot encode channels and offer_type.
- remove underscore in column names.
- rename id to offer id (more meaningful).


In [15]:
import re

def clean_portfolio_data(data):
    """
    This function cleans the portfolio data.
    Channels and offer types are one-hot encoded.
    Column names are made more meaningful.
    return: clean portfolio dataframe.
    """
    data.rename(columns = {'id':'offerid'}, inplace = True) 
    
    # remove underscore
    data.columns = [re.sub('_','', name) for name in data.columns]
    print('Removing underscores in the column names.')
    
    # One hot encode the 'offertype' column
    offertype_df = pd.get_dummies(data['offertype'])
    data = pd.concat([data, offertype_df], axis=1)
    print('One-hot encoding offer types.')
    
    # Create dummy columns for the channels column and drop the original column
    temp = pd.get_dummies(data.channels.apply(pd.Series).stack()).sum(level=0)
    data = pd.concat([data, temp], axis=1, sort=False)
    data.drop(columns='channels', inplace=True)
    print('One-hot encoding channels.')
    
    data = data.drop(columns=['offertype'])
    print('Cleaning the portfolio is now complete.')
    return data

clean_portfolio = clean_portfolio_data(portfolio)

Removing underscores in the column names.
One-hot encoding offer types.
One-hot encoding channels.
Cleaning the portfolio is now complete.


In [16]:
clean_portfolio.head(5)

,difficulty,duration,offerid,reward,bogo,discount,informational,email,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,1,1,1,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0,1
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0,1


#### (b) profile data
- remove null values.
- filter the other gender for simplicity.
- preprocess date column.
- binarize the gender.


In [17]:
import datetime
from sklearn.preprocessing import LabelBinarizer

def clean_profile_data(data):
    """
    This function cleans and transforms the columns of the profile data.
    Null values are removed.
    Other gender is not taken into the account.
    Date column is pre-processed.
    Gneder is binarized.
    return: clean profile dataframe
    """
    # remove customers with null incomes
    data = data[data['income'].isnull()==False]
    print('Null values are removed.')
    
    # keep only two genders for simplicity
    data = data[data['gender'] != 'O']
    print('Other gender is filtered out.')
    
    # reset the index
    data.reset_index(drop=True)
    
    # pre-process date column
    data['became_member_on'] = data['became_member_on'].apply(lambda elem: datetime.datetime.strptime(str(elem), '%Y%m%d'))
    data['startyear'] = data['became_member_on'].apply(lambda elem: elem.year)
    print('Date column is pre-processed.')
    
    # one-hot encode the start year
    tmp = pd.get_dummies(data['startyear'])
    
    # binarize the gender column
    obj = LabelBinarizer()
    data['gender'] = obj.fit_transform(data['gender'])
        
    data = pd.concat([data, tmp], axis=1)
    data = data.drop(columns=['startyear', 'became_member_on'])
    print('Profile data is now cleaned.')
    
    return data

clean_profile = clean_profile_data(profile)

Null values are removed.
Other gender is filtered out.
Date column is pre-processed.
Profile data is now cleaned.


In [18]:
clean_profile.head(5)

,age,gender,customerid,income,2013,2014,2015,2016,2017,2018
1,55,0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,0,0,0,1,0
3,75,0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,0,0,0,1,0
5,68,1,e2127556f4f64592b11af22de27a7932,70000.0,0,0,0,0,0,1
8,65,1,389bc3fa690240e798340f5a15918d5c,53000.0,0,0,0,0,0,1
12,58,1,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,0,0,0,1,0


#### (c) transcript data
- rename person to customerid
- remove all the customre ids which are not in the profile table
- convert hours into days and rename the column
- parse the offers as one-hot encoded columns
- create a separate dataframe for transaction

In [19]:
def make_offer_transaction_data(data):
    """
    This function process the transcript data to contruct offer data and transaction data.
    return: offer data dataframe and transaction dataframe
    """
    # Select customer offers
    obj = re.compile('^offer (?:received|viewed|completed)')

    is_offer = data['event'].apply(lambda elem: obj.match(elem) != None)

    offer_data = data[is_offer].copy()
    offer_data = offer_data.reset_index(drop=True)

    # Create an offerid column
    offer_data['offerid'] = offer_data['value'].apply(lambda elem: list(elem.values())[0])

    # Transform a column that describes a customer offer event
    obj = re.compile('^offer ([a-z]+$)')

    offer_data['event'] = offer_data['event'].apply(lambda elem: obj.match(elem).groups(1)[0])

    # One hot encode customer offer events
    event_df = pd.get_dummies(offer_data['event'])

    # Create a DataFrame that describes customer offer events
    offer_data = pd.concat([offer_data, event_df], axis=1)
    offer_data.drop(columns=['event', 'value'])
    
    #make transaction data
    transaction = data[is_offer == False]
    transaction = transaction.reset_index(drop=True)

    # Transform customer transaction event values
    transaction['amount'] = transaction['value'].apply(lambda elem: list(elem.values())[0])

    # Create a DataFrame that describes customer transactions
    transaction = transaction.drop(columns=['event', 'value'])
    
    return offer_data, transaction


In [20]:
def clean_transcript_data(data):
    """
    This function cleans and tramsfroms the transcript data.
    Rename the columns to be more meaningful.
    Convert the number of hours into number of days for simplicity.
    On-hot encode the relevant columns.
    return: offer_data dataframe and transaction dataframe
    """
    
    # rename the columns
    data.rename(columns = {'person':'customerid'}, inplace = True) 
    
    # remove those customer ids which are not in profile data
    data = data[data['customerid'].isin(clean_profile['customerid'])]
    
    # convert hours to days for easy understanding
    data['time'] /=24
    data = data.rename(columns={'time': 'days'})
    
    offer_data, transaction_data = make_offer_transaction_data(data)
    
    return offer_data, transaction_data

offer_data, transaction_data = clean_transcript_data(transcript)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
transaction_data.head()

,customerid,days,amount
0,02c083884c7d45b39cc68e1314fec56c,0.0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0.0,34.56
2,54890f68699049c2a04d415abc25e717,0.0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,0.0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,0.0,18.97


In [22]:
offer_data.head()

,event,customerid,days,value,offerid,completed,received,viewed
0,received,78afa995795e4d85b5d9ceeca43f5fef,0.0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0
1,received,e2127556f4f64592b11af22de27a7932,0.0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5,0,1,0
2,received,389bc3fa690240e798340f5a15918d5c,0.0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},f19421c1d4aa40978ebb69ca19b0e20d,0,1,0
3,received,2eeac8d8feae4a8cad5a6af0499a211d,0.0,{'offer id': '3f207df678b143eea3cee63160fa8bed'},3f207df678b143eea3cee63160fa8bed,0,1,0
4,received,aa4862eba776480b8bb9c68455b8c2e1,0.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


**Now we have the following processed tables:**
- clean_profile 
- clean_portfolio
- offer_data
- transaction_data

Next we would like to join the above mentioned tables.

In [23]:
# peek into the headers of the processed dataframes
print('profile: ',list(clean_profile.columns))
print('portfolio: ',list(clean_portfolio.columns))
print('offer_data: ',list(offer_data.columns))
print('transaction: ',list(transaction_data.columns))

profile:  ['age', 'gender', 'customerid', 'income', 2013, 2014, 2015, 2016, 2017, 2018]
portfolio:  ['difficulty', 'duration', 'offerid', 'reward', 'bogo', 'discount', 'informational', 'email', 'mobile', 'social', 'web']
offer_data:  ['event', 'customerid', 'days', 'value', 'offerid', 'completed', 'received', 'viewed']
transaction:  ['customerid', 'days', 'amount']


In [24]:
print(clean_profile.shape, clean_portfolio.shape, offer_data.shape, transaction_data.shape)

(14613, 10) (10, 11) (146615, 8) (122176, 3)


In [25]:
clean_data = []

# combining the data to construct meaning
customerid_list = offer_data['customerid'].unique()

for customerid in customerid_list:
    # select profile of the customer
    cur_customer = clean_profile[clean_profile['customerid']==customerid]
    
    # select the associated offers of that customer
    customer_offer_data = offer_data[offer_data['customerid']==customerid]
    customer_offer_data = customer_offer_data.drop(columns='customerid')
    customer_offer_data.reset_index(drop=True)
    
    # select transaction associated to that customer
    customer_transaction_data = transaction_data[transaction_data['customerid']==customerid]
    customer_transaction_data = customer_transaction_data.drop(columns='customerid')
    customer_transaction_data.reset_index(drop=True)
    
    event_type=['completed','received','viewed','value']
    offer_received = customer_offer_data[customer_offer_data['received']==1]
    offer_received = offer_received.drop(columns=event_type).reset_index(drop=True)
    
    offer_viewed = customer_offer_data[customer_offer_data['viewed']==1].drop(columns=event_type).reset_index(drop=True)
    
    offer_completed = customer_offer_data[customer_offer_data['completed']==1].drop(columns=event_type).reset_index(drop=True)

    rows=[]
    for idx in range(offer_received.shape[0]):
        cur_offer_id = offer_received.iloc[idx]['offerid']
        cur_offer = clean_portfolio.loc[clean_portfolio['offerid']==cur_offer_id]
        days = cur_offer['duration'].values[0]
        cur_offer_startime = offer_received.iloc[idx]['days']
        cur_offer_endtime = offer_received.iloc[idx]['days'] + days
        
        #select the transactions which falls in the validity period
        select_transaction = np.logical_and(customer_transaction_data['days']>=cur_offer_startime,
                                           customer_transaction_data['days']<=cur_offer_endtime)
        #select the offers viewed which falls in the validity period
        select_offer_viewed = np.logical_and(offer_viewed['days']>=cur_offer_startime,
                                            offer_viewed['days']<=cur_offer_endtime)
        # select the offers completed which falls in the validity period
        select_offer_completed = np.logical_and(offer_completed['days']>=cur_offer_startime,
                                               offer_completed['days']<=cur_offer_endtime)
        
        # determine whether the curent offer was successful
        cur_offer_successful = select_offer_completed.sum() > 0 and select_offer_viewed.sum()>0
        
        cur_offer_transaction = customer_transaction_data[select_transaction]
        
        cur_row={
            'offerid': cur_offer_id,
            'customerid': customerid,
            'time':cur_offer_startime,
            'offersuccessful':int(cur_offer_successful),
            'totalamount':cur_offer_transaction['amount'].sum()
        }
       
        cur_row.update(cur_offer.iloc[0,1:].to_dict())

        cur_row.update(cur_customer.iloc[0,1:].to_dict())


        # Update a list of dictionaries that describes the effectiveness of 
        # offers to a specific customer
        rows.append(cur_row)
    clean_data.extend(rows)
    
    
clean_data = pd.DataFrame(clean_data)
clean_data.head()

    

,bogo,customerid,discount,duration,email,gender,income,informational,mobile,offerid,...,social,time,totalamount,web,2013,2014,2015,2016,2017,2018
0,1,78afa995795e4d85b5d9ceeca43f5fef,0,7,1,0,100000.0,0,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,0,0.0,37.67,1,0,0,0,0,1,0
1,0,78afa995795e4d85b5d9ceeca43f5fef,0,3,1,0,100000.0,1,1,5a8bc65990b245e5a138643cd4eb9837,...,1,7.0,49.39,0,0,0,0,0,1,0
2,1,78afa995795e4d85b5d9ceeca43f5fef,0,7,1,0,100000.0,0,1,ae264e3637204a6fb9bb56bc8210ddfd,...,1,17.0,48.28,0,0,0,0,0,1,0
3,1,78afa995795e4d85b5d9ceeca43f5fef,0,5,1,0,100000.0,0,1,f19421c1d4aa40978ebb69ca19b0e20d,...,1,21.0,48.28,1,0,0,0,0,1,0
4,0,e2127556f4f64592b11af22de27a7932,1,7,1,1,70000.0,0,1,2906b810c7d4411798c6938adc9daaa5,...,0,0.0,0.00,1,0,0,0,0,0,1


In [26]:
clean_data.columns

Index([           'bogo',      'customerid',        'discount',
              'duration',           'email',          'gender',
                'income',   'informational',          'mobile',
               'offerid', 'offersuccessful',          'reward',
                'social',            'time',     'totalamount',
                   'web',              2013,              2014,
                    2015,              2016,              2017,
                    2018],
      dtype='object')

In [27]:
clean_data = clean_data.sort_values('time')

In [28]:
# clean_data.to_csv('./data/clean_data.csv')

In [29]:
pd.read_csv('./data/clean_data.csv')

,Unnamed: 0,bogo,customerid,discount,duration,email,gender,income,informational,mobile,...,social,time,totalamount,web,2013,2014,2015,2016,2017,2018
0,0,1,78afa995795e4d85b5d9ceeca43f5fef,0,7,1,0,100000.0,0,1,...,0,0.0,37.67,1,0,0,0,0,1,0
1,11262,1,ff7cb44e72db4112b270560686f97a23,0,5,1,0,39000.0,0,1,...,1,0.0,48.31,1,0,0,1,0,0,0
2,35019,0,97b6993c232946d3b6b9f90530ff8073,0,3,1,1,52000.0,1,1,...,1,0.0,23.43,0,0,0,0,0,1,0
3,35016,1,81848348d5584aef9e7374a07ebe6ea1,0,7,1,0,118000.0,0,1,...,0,0.0,52.24,1,0,0,0,1,0,0
4,11266,0,28f9666945804ab0bfc63f3ec6ae9af1,1,10,1,0,44000.0,0,1,...,1,0.0,5.12,1,0,0,0,0,0,1
5,35011,0,5180d955d076408dac0e7a3004756889,0,4,1,1,49000.0,1,1,...,0,0.0,9.99,1,0,0,0,0,1,0
6,11270,1,e4d95c80a1914492aff749a61bee8380,0,5,1,1,85000.0,0,1,...,1,0.0,0.00,1,0,0,0,0,0,1
7,35007,1,47621e81a9bc41459044c0ed409dfa9e,0,5,1,1,73000.0,0,1,...,1,0.0,6.05,1,0,0,0,0,1,0
8,35002,0,f726eb45d69845b9b5b8100d074c64f7,1,7,1,1,58000.0,0,1,...,0,0.0,47.78,1,0,0,0,0,1,0
9,34998,1,331597393f414b0285918854d8ac292a,0,7,1,0,75000.0,0,1,...,0,0.0,0.00,1,0,0,0,0,1,0
